In [1]:
!pip install openai python-dotenv


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
# download sdxl & ipadapter
base_model_path = "stabilityai/stable-diffusion-xl-base-1.0"
image_encoder_path = "models/image_encoder.safetensors"
ip_ckpt = "models/sdxl_ipadapter_plus_face.safetensors"

!mkdir models
!wget -O {ip_ckpt} https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors
!wget -O {image_encoder_path} https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors

--2023-11-17 10:13:29--  https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 52.84.90.33, 52.84.90.129, 52.84.90.122, ...
Connecting to huggingface.co (huggingface.co)|52.84.90.33|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/7f/2f/7f2fe2e27137549cd28e570e5bac269b49ebcf1e0e47279c7a941ebe5c948e02/31e35c80fc4829d14f90153f4c74cd59c90b779f6afe05a74cd6120b893f7e5b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sd_xl_base_1.0.safetensors%3B+filename%3D%22sd_xl_base_1.0.safetensors%22%3B&Expires=1700472889&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMDQ3Mjg4OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83Zi8yZi83ZjJmZTJlMjcxMzc1NDljZDI4ZTU3MGU1YmFjMjY5YjQ5ZWJjZjFlMGU0NzI3OWM3YTk0MWViZTVjOTQ4ZTAyLzMxZTM1YzgwZmM0ODI5ZDE0ZjkwMTUz

In [2]:
import dotenv
import openai
import re
dotenv.load_dotenv('../.env')

True

In [56]:
hero_name = "Léa"
story_idea = "une jeune sorcière en formation qui doit prouver ses compétences en sauvant son village d'un sortilège maléfique"
instructions = """
Consignes:
Un chapitre doit être court et se terminer par un cliffhanger.
Un Chapitre est divisé en plusieurs court paragraphes.
Un paragraphes contient Deux phrases au maximum."""
imagen_instructions = """-----
Write a very short one sentence visual description of the scene. Include a very short description of the character and its environnement. Keep it short and simple."""

In [12]:
def generate_prompt(story_idea, hero_name, instructions):
    prompt = f"""Ecris une histoire pour enfant composée de 5 chapitres.
{story_idea}
Personnage principal: {hero_name}
{instructions}
"""
    return prompt

In [13]:
def generate_story(story_idea, hero_name, instructions):
    prompt = generate_prompt(story_idea, hero_name, instructions)
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user", "content": prompt}]
    )
    full_text = response.choices[0].message.content
    return full_text

In [14]:
story = generate_story(story_idea, hero_name, instructions)
print(story)

Chapitre 1: Les débuts de Léa

Léa, une jeune sorcière en formation, vivait dans un petit village paisible entouré de forêts enchantées. Depuis toute petite, elle rêvait de devenir une grande sorcière et d'aider les habitants de son village avec ses pouvoirs magiques. Un jour, son plus grand rêve se réalisa lorsqu'elle fut acceptée à l'Académie des Sorcières. Elle quitta son village avec enthousiasme, les yeux remplis d'étoiles. Mais à peine arrivée, elle fut confrontée à un défi inattendu : un sortilège maléfique enveloppa son village natal, plongeant ses habitants dans le désespoir. Léa savait qu'elle devait revenir et sauver son village coûte que coûte.

Chapitre 2: La quête de Léa

Déterminée à sauver son village, Léa commença sa quête. Elle se rendit dans la bibliothèque de l'Académie où elle étudia les sorts et les enchantements pour contrer le sortilège maléfique. Elle découvrit qu'elle devait réunir sept ingrédients magiques dispersés dans le royaume enchanté, afin de créer un 

In [33]:

def format_raw_story(story):

    # split story into chapters
    pattern = r"(Chapitre \d+: [^\n]+)\n\n(.*?)(?=\n\nChapitre|$)"

    # Find all matches
    story_by_chapters = re.findall(pattern, story, re.DOTALL)

    # Split the chapters into paragraphs of at least 100 characters
    story_by_chapters_and_paragraphs = []
    for chapter in story_by_chapters:
        chapter_title = chapter[0]
        chapter_content = chapter[1]
        chapter_content_splited = []
        for sentence in chapter_content.split("."):
            sentence = sentence.strip()
            if sentence == "": continue
            if len(chapter_content_splited) == 0:
                chapter_content_splited.append(sentence)
            elif len(chapter_content_splited[-1]) < 100:
                chapter_content_splited[-1] += f". {sentence}"
            else:
                chapter_content_splited.append(sentence)
        story_by_chapters_and_paragraphs.append((chapter_title, chapter_content_splited))
    
    return story_by_chapters_and_paragraphs

In [34]:
formated_story = format_raw_story(story)

In [60]:
formated_story

[('Chapitre 1: Les débuts de Léa',
  ['Léa, une jeune sorcière en formation, vivait dans un petit village paisible entouré de forêts enchantées',
   "Depuis toute petite, elle rêvait de devenir une grande sorcière et d'aider les habitants de son village avec ses pouvoirs magiques",
   "Un jour, son plus grand rêve se réalisa lorsqu'elle fut acceptée à l'Académie des Sorcières. Elle quitta son village avec enthousiasme, les yeux remplis d'étoiles",
   'Mais à peine arrivée, elle fut confrontée à un défi inattendu : un sortilège maléfique enveloppa son village natal, plongeant ses habitants dans le désespoir',
   "Léa savait qu'elle devait revenir et sauver son village coûte que coûte"]),
 ('Chapitre 2: La quête de Léa',
  ["Déterminée à sauver son village, Léa commença sa quête. Elle se rendit dans la bibliothèque de l'Académie où elle étudia les sorts et les enchantements pour contrer le sortilège maléfique",
   "Elle découvrit qu'elle devait réunir sept ingrédients magiques dispersés 

In [61]:
def text_to_images_desc(chapter_content):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user", "content": f"""{chapter_content}
{imagen_instructions}"""}
                  ],
        max_tokens=50
    )
    full_text = response.choices[0].message.content
    return full_text

                   
    

In [62]:
story_with_images_desc = dict()
for i, chapter in enumerate(formated_story):
    chapter_title = chapter[0]
    chapter_content = chapter[1]
    story_with_images_desc[f'chapter_{i}'] = dict()
    story_with_images_desc[f'chapter_{i}']['title'] = chapter_title
    print(chapter_title)
    for j, paragraph in enumerate(chapter_content):
        paragraph_image_desc = text_to_images_desc(paragraph)
        story_with_images_desc[f'chapter_{i}'][f'paragraph_{j}'] = dict()
        story_with_images_desc[f'chapter_{i}'][f'paragraph_{j}']['text'] = paragraph
        story_with_images_desc[f'chapter_{i}'][f'paragraph_{j}']['image_desc'] = paragraph_image_desc

story_with_images_desc
        
        


Chapitre 1: Les débuts de Léa
Chapitre 2: La quête de Léa
Chapitre 3: Les rencontres magiques de Léa
Chapitre 4: Le piège diabolique
Chapitre 5: La victoire de Léa


{'chapter_0': {'title': 'Chapitre 1: Les débuts de Léa',
  'paragraph_0': {'text': 'Léa, une jeune sorcière en formation, vivait dans un petit village paisible entouré de forêts enchantées',
   'image_desc': 'Léa, a young witch-in-training, resided in a serene village nestled amidst enchanting forests.'},
  'paragraph_1': {'text': "Depuis toute petite, elle rêvait de devenir une grande sorcière et d'aider les habitants de son village avec ses pouvoirs magiques",
   'image_desc': 'A young girl, surrounded by a quaint village, dreams of becoming a powerful witch to help its inhabitants.'},
  'paragraph_2': {'text': "Un jour, son plus grand rêve se réalisa lorsqu'elle fut acceptée à l'Académie des Sorcières. Elle quitta son village avec enthousiasme, les yeux remplis d'étoiles",
   'image_desc': 'A young girl, with excitement shining in her eyes, joyfully leaves her village as she sets off to the Witch Academy.'},
  'paragraph_3': {'text': 'Mais à peine arrivée, elle fut confrontée à un d

# Gen images

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline
from PIL import Image

from ip_adapter import IPAdapterXL

In [ ]:
# load SDXL pipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    add_watermarker=False,
)

In [ ]:
# load ip-adapter
ip_model = IPAdapterXL(pipe, image_encoder_path, ip_ckpt, "cuda")


In [ ]:
# read image prompt
image = Image.open("assets/images/woman.png")
image.resize((512, 512))

In [ ]:
images = ip_model.generate(pil_image=image, num_samples=1, num_inference_steps=30,
                           prompt="best quality, high quality, wearing sunglasses on the beach", scale=0.6)
